In [1]:
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
# import weaviate
# import json
import torch
from langchain.text_splitter import CharacterTextSplitter
from transformers import BertTokenizer, BertModel, pipeline
import requests
import time


/Users/orenciolli/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
url = "https://www.npr.org/sections/news/archive?date=12-31-2020"

In [61]:
texts = []
    
response = requests.get(url)

#scraping base articles
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    
    archive_list = soup.find('div', class_='archivelist')

    articles = archive_list.find_all('article', recursive=False)

    for article in articles:

        link = article.find('a')['href']
        
        article_response = requests.get(link)
        
        if article_response.status_code == 200:
            
            article_soup = BeautifulSoup(article_response.content, "html.parser")
            
            paragraphs = article_soup.find_all('p')

    
            #instead of appending here, just embed and post to db
            text = '\n'.join([p.get_text() for p in paragraphs])
            texts.append(text)
        
        else:
            print(f"Failed to fetch {link}. Status code: {article_response.status_code}")

# Infinite scroll

In [3]:
from selenium import webdriver

In [4]:
def extract_articles(soup):
    articles = []
    archive_list = soup.find('div', {'id': 'infinitescroll'})
    if archive_list:
        articles.extend(archive_list.find_all('article', recursive=False))
    return articles

def extract_text_from_article(article):
    link = article.find('a')['href']
    article_response = requests.get(link)
    if article_response.status_code == 200:
        article_soup = BeautifulSoup(article_response.content, "html.parser")
        paragraphs = article_soup.find_all('p')
        text = '\n'.join([p.get_text() for p in paragraphs])
        return text
    else:
        print(f"Failed to fetch {link}. Status code: {article_response.status_code}")
        return None

def simulate_scroll(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(2)

def is_loading_complete(soup):
    loading_div = soup.find('div', id='infscr-loading')
    #the div is set to display: none after ~4 seconds, but it takes longer than that to scrape
    return loading_div is None or 'display: none;' in loading_div.get('style', '')


visited_links = set()

In [23]:
driver = webdriver.Chrome()  

driver.get(url)

In [18]:
from selenium.webdriver.common.by import By

In [26]:
simulate_scroll(driver)
driver.find_element(By.CLASS_NAME, "pn-modal__close").click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pn-modal__close"}
  (Session info: chrome=121.0.6167.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000101891538 chromedriver + 4687160
1   chromedriver                        0x0000000101888d83 chromedriver + 4652419
2   chromedriver                        0x0000000101479fbd chromedriver + 397245
3   chromedriver                        0x00000001014c5c3c chromedriver + 707644
4   chromedriver                        0x00000001014c5e11 chromedriver + 708113
5   chromedriver                        0x000000010150a274 chromedriver + 987764
6   chromedriver                        0x00000001014e892d chromedriver + 850221
7   chromedriver                        0x00000001015077bc chromedriver + 976828
8   chromedriver    

In [14]:
i = 0
while i < 1:
   
    simulate_scroll(driver)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = extract_articles(soup)
    
    
    
    for article in articles:
        link = article.find('a')['href']
        
#         if link not in visited_links:
#             text = extract_text_from_article(article)
#             if text:
#                 texts.append(text)
#                 visited_links.add(link)
    
    
    simulate_scroll(driver)
    i += 1
    
   
    if is_loading_complete(soup):
        print('yeah')

        
        
# driver.quit()

yeah


In [15]:
driver.quit()

In [72]:
texts

['\n\n      Vanessa Romo\n    \n\n\n                The Florida Department of Health said officials are working alongside the Centers for Disease Control and Prevention to figure out how a 20-something-year-old man contracted the new, more contagious coronavirus strain that was first discovered in the U.K.\n                \n                    \n                    Lynne Sladky/AP\n                    \n                \nhide caption\n\nThe Florida Department of Health said officials are working alongside the Centers for Disease Control and Prevention to figure out how a 20-something-year-old man contracted the new, more contagious coronavirus strain that was first discovered in the U.K.\nFlorida is the third U.S. state to announce it has a case of the more contagious coronavirus strain that first emerged in the United Kingdom.\nA man in his 20s, with no history of travel, tested positive for the mutated coronavirus. The state Department of Health said he is in Martin County.\nFlorida